In [ ]:
import cv2
import matplotlib.pyplot as plt

In [ ]:
img = cv2.imread("cat.jpg")
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

In [ ]:
img.shape

In [ ]:
plt.imshow(img)

In [ ]:
plt.imshow(img)

In [1]:
import os

In [ ]:
os.mkdir("CatsAndDogs")
os.mkdir("CatsAndDogs/train")
os.mkdir("CatsAndDogs/test")

In [ ]:
categories = ['cat', 'dog']

for category in categories:
    os.mkdir("CatsAndDogs/train/"+category)
    os.mkdir("CatsAndDogs/test/"+category)

In [ ]:
data = os.listdir('train/train')

In [ ]:
cats_count, dogs_count = 0,0
for file_name in data:
    if 'cat' in file_name:
        cats_count+=1
    else:
        dogs_count+=1
print(cats_count, dogs_count)

In [ ]:
from sklearn.model_selection import train_test_split

# split into train test sets
train, test = train_test_split(list(range(0,len(data)//2)))

In [ ]:
import shutil
from tqdm import tqdm

for file_name in tqdm(data):
    category, image_id, file_type = file_name.split(".")
    if int(image_id) in train:
        shutil.move("train/train/"+file_name, "CatsAndDogs/train/"+category+"/"+file_name)
    else:
        shutil.move("train/train/"+file_name, "CatsAndDogs/test/"+category+"/"+file_name)

In [2]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_generator = ImageDataGenerator(rescale=1./255,
                                     zoom_range=0.1,
                                     shear_range=0.1,
                                     horizontal_flip=True)
test_generator = ImageDataGenerator(rescale=1./255)

In [8]:
INPUT_SHAPE = (240,240)
BATCH_SIZE = 1000
EPOCHS = 100

train_data = train_generator.flow_from_directory("CatsAndDogs/train/",
                                                target_size = INPUT_SHAPE,
                                                batch_size = BATCH_SIZE,
                                                class_mode = 'binary')
test_data = test_generator.flow_from_directory("CatsAndDogs/test/",
                                                target_size = INPUT_SHAPE,
                                                batch_size = BATCH_SIZE,
                                                class_mode = 'binary')

Found 18750 images belonging to 2 classes.
Found 6250 images belonging to 2 classes.


In [12]:
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Conv2D, MaxPool2D, Activation, Flatten, Dense, Dropout

model = Sequential()
model.add(Conv2D(32, (3,3), input_shape=(240,240,3)))
model.add(Activation('relu'))
model.add(MaxPool2D(pool_size=(2,2)))

model.add(Conv2D(32, (3,3)))
model.add(Activation("relu"))
model.add(MaxPool2D(pool_size=(2,2)))

model.add(Conv2D(32, (3,3)))
model.add(Activation("relu"))
model.add(MaxPool2D(pool_size=(2,2)))

model.add(Flatten())
model.add(Dense(50))
model.add(Activation('relu'))
model.add(Dropout(0.4))

model.add(Dense(15))
model.add(Activation('relu'))
model.add(Dropout(0.4))

model.add(Dense(1))
model.add(Activation('sigmoid'))

In [13]:
model.compile(loss='binary_crossentropy',
             optimizer='rmsprop',
             metrics='accuracy')

In [14]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_4 (Conv2D)            (None, 238, 238, 32)      896       
_________________________________________________________________
activation_6 (Activation)    (None, 238, 238, 32)      0         
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 119, 119, 32)      0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 117, 117, 32)      9248      
_________________________________________________________________
activation_7 (Activation)    (None, 117, 117, 32)      0         
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 58, 58, 32)        0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 56, 56, 32)       

In [ ]:
hist = model.fit(train_data,
                steps_per_epoch=len(train_data.filenames)//BATCH_SIZE,
                epochs = EPOCHS, 
                validation_data = test_data,
                validation_steps = len(test_data.filenames)//BATCH_SIZE,
                )

Epoch 1/100
